In [1]:
!pip install pip install autogluon

  Using cached install-1.3.5-py3-none-any.whl (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 9.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 6.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 3.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.4 MB/s

In [2]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

ModuleNotFoundError: No module named 'autogluon'

In [15]:
train_df = pd.read_csv('train.csv')[['ID','timestamp','supply(kg)', 'price(원/kg)']]
# test_df = pd.read_csv('test.csv')[['ID','timestamp']]

In [16]:
train_df['item_id'] = train_df.ID.str[0:6]
# test_df['item_id'] = test_df.ID.str[0:6]

In [17]:
data = TimeSeriesDataFrame(train_df.drop(columns=['ID']))
predictor = TimeSeriesPredictor( 
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)
# seed 고정
predictor.fit( data, random_seed=42, )

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
Global seed set to 42
AutoGluon will save models to AutogluonModels\ag-20231112_052345\
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)']

Starting training. Start time is 202

In [18]:
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	2.19    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 2.40 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [19]:
# seed 고정
pred = predictor.predict(data, random_seed=42, )

Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [20]:
pred

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3249.371428  1061.962512  1822.099598  2356.666160   
        2023-03-05   685.749831 -2060.506732 -1122.077911  -447.970651   
        2023-03-06  3044.917527  -480.330324   808.741333  1657.877709   
        2023-03-07  3463.382329  -435.830595   925.919328  1858.888452   
        2023-03-08  3304.855823  -856.234747   587.264285  1612.549541   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   502.113238  -220.864280    57.778526   225.951939   
        2023-03-28   521.814296  -230.533786    68.465458   238.048419   
        2023-03-29   519.017230  -272.460916    47.440414   233.578369   
        2023-03-30   506.832145  -264.524065    37.183076   226.988193   
        2023-03-31   511.823768  -297.308532    18.280521   213.199803   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  2831.553078  3274.046010  3707.234661  4172.362595   
        2023-03-05   117.590776   638.844024  1181.768701  1752.230389   
        2023-03-06  2397.617743  3046.794298  3698.106493  4405.636461   
        2023-03-07  2645.802365  3413.735307  4185.687008  4994.269255   
        2023-03-08  2489.554600  3296.153781  4110.799357  5002.118203   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   373.761455   506.834047   632.832728   771.448217   
        2023-03-28   386.681816   528.159235   671.685615   814.718084   
        2023-03-29   384.538602   531.467662   668.278763   818.807452   
        2023-03-30   372.222564   509.844633   647.372533   796.298493   
        2023-03-31   368.650540   511.401207   659.188667   818.049213   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4714.482384  5514.882444  
        2023-03-05  2428.259075  3363.004906  
        2023-03-06  5237.350182  6492.382986  
        2023-03-07  5956.045761  7290.270292  
        2023-03-08  6032.298060  7480.064991  
...                         ...          ...  
RD_F_J  2023-03-27   938.181708  1187.946035  
        2023-03-28   983.599501  1224.211551  
        2023-03-29   992.982668  1232.181649  
        2023-03-30   969.568536  1215.261210  
        2023-03-31   992.375615  1230.679636  

[1092 rows x 10 columns]

In [21]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission.to_csv('./dacon_submission.csv', index=False)
submission

,ID,answer
0,TG_A_J_20230304,3249.371428
1,TG_A_J_20230305,685.749831
2,TG_A_J_20230306,3044.917527
3,TG_A_J_20230307,3463.382329
4,TG_A_J_20230308,3304.855823
...,...,...
1087,RD_F_J_20230327,502.113238
1088,RD_F_J_20230328,521.814296
1089,RD_F_J_20230329,519.017230
1090,RD_F_J_20230330,506.832145
